# Projeto de Aprendizagem Automática II

## Procura de Exoplanetas no Espaço através da Emissão de Luz de Estrelas

### Importação de Bibliotecas

In [1]:
import pandas as pd 
import numpy as np  

import matplotlib            
import matplotlib.pyplot as plt

import seaborn as sns        
color = sns.color_palette()
sns.set_style('darkgrid')

import os

from random import randint
from scipy.stats import randint as sp_randint

from time import time
from datetime import datetime

import sklearn               
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, VarianceThreshold, chi2

from warnings import simplefilter

%matplotlib inline
simplefilter(action='ignore', category=FutureWarning)

### Carregamento dos Dados

In [2]:
treino = pd.read_csv("../../../../../Dados/dados_treino.csv")
teste = pd.read_csv("../../../../../Dados/dados_teste.csv")

O primeiro passo é a divisão entre *labels* e atributos, tanto no conjunto de dados de treino, como no conjunto de dados de teste.

In [3]:
X_train = treino.loc[:, treino.columns != 'LABEL'].values
Y_train = treino.LABEL.values

X_test = teste.loc[:, teste.columns != 'LABEL'].values
Y_test = teste.LABEL.values

parameters = {
    'n_neighbors':[3, 5, 7],
    'weights': ['uniform', 'distance'],
}

Em seguida, tendo em conta a fácil forma como se efetua uma procura em grelha neste tipo de algoritmos, vão ser testados 3, 5 e 7 como os possíveis números de vizinhos e os pesos como uniformes ou distâncias.

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

clf = GridSearchCV(KNeighborsClassifier(), parameters, cv=3, verbose=1)
clf.fit(X_train, Y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:  6.3min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [3, 5, 7],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

Analisando os resultados quer do modelo, quer da pesquisa em grelha, é possível observar que para os parâmetros testados, os melhores resultados foram obtidos com 3 vizinhos e pesos uniformes, sendo obtido um *score* na casa dos 88%.

In [5]:
print(clf.best_params_)
print(clf.best_estimator_)
print(clf.best_score_)

{'n_neighbors': 3, 'weights': 'uniform'}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')
0.8763137852246763


Aplicando o modelo ao conjunto de treino, é notório que quer a precisão, quer o *recall* são bastante baixos.

In [8]:
from sklearn.metrics import classification_report
y_pred = clf.predict(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           1       0.99      1.00      0.99       565
           2       0.33      0.20      0.25         5

    accuracy                           0.99       570
   macro avg       0.66      0.60      0.62       570
weighted avg       0.99      0.99      0.99       570



De modo a obter uma visão mais simplificada dos resultados, tenha-se a matriz de confusão. Apenas um sistema com exoplanetas foi corretamente classificado, mostrando que este algoritmo, com as configurações em questão não produz resultados muito promissores.

In [9]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test, y_pred))

[[563   2]
 [  4   1]]


### PCA

Numa tentiva de melhorar os resultados, bem como diminuir o peso computacional, a ideia passou pela aplicação de *PCA*. Assim, mantendo 90% da explicação da variável de decisão com base nas *features*, a dimensionalidade do conjunto de dados foi reduzida para 125 colunas por linha.

In [4]:
from sklearn.decomposition import PCA
pca = PCA(0.90, True, False)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
print("the dimension of decomposed data is ", X_train.shape[1])

the dimension of decomposed data is  125


Utilizando o melhor modelo da procura em grelha, obtido anteriormente, ou seja, a métrica de distância selecionada foi *Minkowski*, com pesos uniformes e 3 vizinhos por registo.

In [7]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_jobs=None, n_neighbors=3, p=2, weights='uniform')
model.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

### Teste

Pela análise das métricas, é notório que esta versão com *PCA* mostrou resultados piores em comparação à anterior. Classificando de forma errada todos os sistemas com exoplanetas no conjunto de teste. 

In [9]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           1       0.99      1.00      0.99       565
           2       0.00      0.00      0.00         5

    accuracy                           0.99       570
   macro avg       0.50      0.50      0.50       570
weighted avg       0.98      0.99      0.99       570



A análise da matriz de confusão permite comprovar a classificação errada de todos os sistemas com exoplanetas. Além disso, classifica como possuindo exoplanetas dois sistemas de forma errada. 

In [10]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test, y_pred))

[[563   2]
 [  5   0]]


Pela análise dos dois modelos testados, este tipo de algoritmo não se mostra apropriado para o problema em questão, podendo isto dever-se à complexidade dos dados ou ao tipo de métricas de distância entre registos, já que não são próprias para séries temporais.